In [1]:
# load libraries for data manipulation
import numpy as np
import pandas as pd

# Read in data

In [2]:
# load language dataset - this one is pretty clean
lang_raw = pd.read_csv(r"data_in/SA1_2016_LANP.csv")
print(lang_raw.shape) # the shape of the dataset
lang_raw.head(5) # the first five rows

(3985, 4)


,Counting,SA1 (UR),LANP - 4 Digit Level,Count
0,"Persons, Place of Usual Residence",2111302,English,283
1,"Persons, Place of Usual Residence",2111302,French,5
2,"Persons, Place of Usual Residence",2111302,Greek,9
3,"Persons, Place of Usual Residence",2111302,Spanish,5
4,"Persons, Place of Usual Residence",2111302,Italian,17


In [3]:
# household composition - this one has extra rows on the top and bottom that needs to be deleted 
# this has been downloaded from tablebuilder
hhcd_raw = pd.read_csv(r"data_in/SA1_2016_hhcd2.csv", skiprows = 9) 
hhcd_raw = hhcd_raw[hhcd_raw['Count'].notnull()] # drop the na values
hhcd_raw = hhcd_raw[~(hhcd_raw['HHCD - 2 Digit Level'] == "Not applicable")] # filter out the not applicables
hhcd_raw = hhcd_raw.drop(columns = 'Unnamed: 4') # drop this
print(hhcd_raw.shape) # the shape of the dataset
hhcd_raw.head(5)

(2144, 4)


,Counting,SA1,HHCD - 2 Digit Level,Count
0,Dwellings Location on Census Night,2111301,One family household with only family members ...,0.0
1,Dwellings Location on Census Night,2111301,One family household with non-family members p...,0.0
2,Dwellings Location on Census Night,2111301,Two family household,0.0
3,Dwellings Location on Census Night,2111301,Three or more family household,0.0
4,Dwellings Location on Census Night,2111301,Lone person household,0.0


# Manipulate data

## Languages spoken at home

In [4]:
lang = lang_raw.drop(columns = 'Counting') # drop a column
lang.columns = ['sa1', 'lanp_4_digit_level', 'count'] # rename columns

# filter out 
lang_remove = ['English', 'Not stated'] # remove these
lang = lang[~lang['lanp_4_digit_level'].isin(lang_remove)]

# sort values
lang = lang.sort_values(['sa1', 'count'], ascending=[True, False])
lang.head(5)

,sa1,lanp_4_digit_level,count
4,2111302,Italian,17
13,2111302,Cantonese,11
2,2111302,Greek,9
7,2111302,Arabic,9
5,2111302,Maltese,8


Get the top language for each `sa1`.

In [5]:
lang_1 = lang.groupby('sa1').first() # this was already sorted
lang_1.head(5)

,lanp_4_digit_level,count
sa1,,
2111302,Italian,17
2111303,Italian,33
2111304,Italian,35
2111305,Italian,34
2111306,Italian,21


In [6]:
lang_2 = lang.groupby('sa1').nth(1) # the second row. Python indexing starts at zero
lang_2.head(5)

,lanp_4_digit_level,count
sa1,,
2111302,Cantonese,11
2111303,Vietnamese,14
2111304,Greek,11
2111305,Mandarin,23
2111306,Greek,12


In [7]:
lang_3 = lang.groupby('sa1').nth(2) # the third row. Python indexing starts at zero
lang_3.head(5)

,lanp_4_digit_level,count
sa1,,
2111302,Greek,9
2111303,Spanish,10
2111304,Vietnamese,7
2111305,Spanish,20
2111306,Mandarin,10


In [8]:
lang_top3 = lang_1.join(lang_2, on = 'sa1', lsuffix = '1', rsuffix = '2') # prefix and suffix needed for cols
lang_top3 = lang_top3.join(lang_3, on = 'sa1', rsuffix = '3') # prefix and suffix needed for cols
lang_top3 = lang_top3.reset_index() # reset the index
lang_top3.columns = ['sa1', 'lang1', 'lang1_count', 'lang2', 'lang2_count', 'lang3', 'lang3_count'] # rename columns
lang_top3['sa1'] = lang_top3['sa1'].astype(object) # convert from integer to object
lang_top3.to_csv("data_out/lang_top3_2016_pd.csv", index=False) # export without the index
lang_top3.head(5)

,sa1,lang1,lang1_count,lang2,lang2_count,lang3,lang3_count
0,2111302,Italian,17,Cantonese,11,Greek,9
1,2111303,Italian,33,Vietnamese,14,Spanish,10
2,2111304,Italian,35,Greek,11,Vietnamese,7
3,2111305,Italian,34,Mandarin,23,Spanish,20
4,2111306,Italian,21,Greek,12,Mandarin,10


## Lone person households

In [9]:
lone_pers_hh = hhcd_raw.drop(columns = 'Counting') # drop col
lone_pers_hh.columns = ['sa1', 'lone_pers_hh', 'count'] # rename columns

# percentages
f = lambda x: 100 * x / float(x.sum()) # function to pass in below
lone_pers_hh['pct'] = round((lone_pers_hh.groupby(['sa1'])['count'].transform(f)), 1) # new column
lone_pers_hh = lone_pers_hh[lone_pers_hh['pct'].notnull()] # drop the na values
lone_pers_hh = lone_pers_hh[(lone_pers_hh['lone_pers_hh']) == "Lone person household"] # just the lone person households
lone_pers_hh['sa1'] = lone_pers_hh['sa1'].astype(object) # convert from integer to object
lone_pers_hh.to_csv('data_out/lone_pers_hh_sa1_2016_pd.csv', index=False) # export without the index
lone_pers_hh.head(5)

,sa1,lone_pers_hh,count,pct
13,2111302,Lone person household,48.0,27.6
22,2111303,Lone person household,40.0,19.2
31,2111304,Lone person household,55.0,24.1
40,2111305,Lone person household,79.0,23.8
49,2111306,Lone person household,45.0,19.0


# Spatial joins

If joining in ArcGIS then you could stop here. Otherwise, we can use `geopandas`

In [10]:
import geopandas as gpd

In [11]:
sa1_shp_raw = gpd.read_file("data_in/shp/SA1_2016_MV.shp") 
sa1_cols_sel = ['SA1_7DIGIT', 'geometry'] # cols to keep
sa1_shp = sa1_shp_raw[sa1_cols_sel] # select them
sa1_shp.columns = ['sa1', 'geometry'] # rename columns
sa1_shp.head(5)

,sa1,geometry
0,2111301,"POLYGON ((314804.0597000001 5816681.2025, 3148..."
1,2111302,"POLYGON ((315389.9187000003 5817385.5291, 3152..."
2,2111303,"POLYGON ((315057.0010000002 5817498.3361, 3150..."
3,2111304,"POLYGON ((315800.2566999998 5816719.4447, 3158..."
4,2111305,"POLYGON ((315406.4104000004 5815778.8661, 3154..."


In [12]:
sa1_shp.crs # the crs

{'init': 'epsg:28355'}

In [13]:
sa1_shp.dtypes

sa1         object
geometry    object
dtype: object

## Lone person households

In [14]:
sa1_lone_pers_shp = sa1_shp.merge(lone_pers_hh, on = 'sa1', how = 'left')
sa1_lone_pers_shp = sa1_lone_pers_shp[sa1_lone_pers_shp['count'].notnull()] # drop the na values
sa1_lone_pers_shp.to_file("data_out/shp/lone_pers_hh_gpd.shp") # export shapefile
sa1_lone_pers_shp.head(5)

,sa1,geometry,lone_pers_hh,count,pct
1,2111302,"POLYGON ((315389.9187000003 5817385.5291, 3152...",Lone person household,48.0,27.6
2,2111303,"POLYGON ((315057.0010000002 5817498.3361, 3150...",Lone person household,40.0,19.2
3,2111304,"POLYGON ((315800.2566999998 5816719.4447, 3158...",Lone person household,55.0,24.1
4,2111305,"POLYGON ((315406.4104000004 5815778.8661, 3154...",Lone person household,79.0,23.8
5,2111306,"POLYGON ((315204.1069 5816705.8674, 315303.790...",Lone person household,45.0,19.0


In [15]:
sa1_lone_pers_shp.crs

{'init': 'epsg:28355'}

## Top three languages

In [16]:
# not sure what is going on here. not quite working
sa1_lang_top3_shp = sa1_shp.merge(lang_top3, on = 'sa1', how = 'left')
sa1_lang_top3_shp.head(5)

,sa1,geometry,lang1,lang1_count,lang2,lang2_count,lang3,lang3_count
0,2111301,"POLYGON ((314804.0597000001 5816681.2025, 3148...",NaN,NaN,NaN,NaN,NaN,NaN
1,2111302,"POLYGON ((315389.9187000003 5817385.5291, 3152...",NaN,NaN,NaN,NaN,NaN,NaN
2,2111303,"POLYGON ((315057.0010000002 5817498.3361, 3150...",NaN,NaN,NaN,NaN,NaN,NaN
3,2111304,"POLYGON ((315800.2566999998 5816719.4447, 3158...",NaN,NaN,NaN,NaN,NaN,NaN
4,2111305,"POLYGON ((315406.4104000004 5815778.8661, 3154...",NaN,NaN,NaN,NaN,NaN,NaN
